In [14]:
from os import path
import pandas as pd
import numpy as np
import json
import requests
from tqdm import tqdm
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Định nghĩa đường dẫn
DATA_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\data"
RESULT_DIR = r"c:\Users\Admin\Python\ABSA_Prompting\results"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [15]:
load_dotenv()

True

In [16]:
# CONFIG
ViABSA_BP_dir = path.join(DATA_DIR, 'ViABSA_BP')
test_file = path.join(ViABSA_BP_dir, 'data_test.csv')
test_df = pd.read_csv(test_file)

In [17]:
def transform_aspect_sentiment(df, start=0, end=None):
    aspects = [
        "stayingpower",
        "texture",
        "smell",
        "price",
        "others",
        "colour",
        "shipping",
        "packing"
    ]

    if end is None:
        end = len(df)

    result = []

    for idx, row in df.iloc[start:end].iterrows():
        entry = {
            "id": str(idx),
            "text": row['data'],
            "sentiments": []
        }

        for aspect in aspects:
            sentiment = row[f"{aspect}_label"]
            if sentiment == 1: 
                aspect_sentiment_value = row[aspect]
                if aspect_sentiment_value != 'none':
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": aspect_sentiment_value
                    })
                else:
                    entry["sentiments"].append({
                        "aspect": aspect,
                        "sentiment": "unknown"
                    })

        result.append(entry)

    return result

In [18]:
# SETUP DATA
aspects = ['stayingpower', 'texture', 'smell', 'price', 'others', 'colour', 'shipping', 'packing']
test_df[aspects] = test_df[aspects].fillna('none')

for aspect in aspects:
    test_df[aspect + '_label'] = (test_df[aspect] != 'none').astype(int)

test_json = transform_aspect_sentiment(test_df, 0, 100)
test_json[:5]

[{'id': '0',
  'text': 'Hàng đóng gói đẹp và chắc chắn, nhìn rất dễ thương và ưng bụng ạ!',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '1',
  'text': 'Cảm giác son bên trong rất là ít luôn í, đóng gói cẩn thận, dịch nhưng mà giao hàng khá nhanh',
  'sentiments': [{'aspect': 'shipping', 'sentiment': 'positive'},
   {'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '2',
  'text': 'Son siêu đẹp luôn ý, mà y hình mà chụp có thể không giống lắm nhưng nhìn ngoài thì giống nha. Chất son mềm mướt nói chung là rất thíchhhh',
  'sentiments': [{'aspect': 'texture', 'sentiment': 'positive'}]},
 {'id': '3',
  'text': 'Siu đẹp luôn \r\nShop đóng gói cẩn thận lắm luôn \r\nLại thêm cả quà nữa\r\nNói chung là thích lắm',
  'sentiments': [{'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '4',
  'text': 'Aúhihđcyihb gfxxth jj bhgfzđE G GHVHBTCEETXUBBIYCZRZRTCVUUBYRZXGVBIINVUTXZRCTIBONONBUXTZRTVKNNOUVTXEecyuvknibtcrztxuvbubibijvcytdgunonobiyvdtrdfyyghuojpmib

**Ý tưởng chọn example cho few-shot**
1.   Chọn ra những câu đa dạng về aspect từ dataset -> gôm cụm chúng lại để LLMs hiểu rõ hơn về pattern để học
2.   Chọn ra những câu khó, những câu phức tạp hơn 

Áp dụng K-Means Clustering + Chọn Centroid + Sampling Example khó

In [19]:
'''
    Nếu dùng TD-IDF thì sẽ chỉ chọn dựa vào tần suất từ, không hiểu nghĩa của câu -> bị trùng lặp
    Semantic grouping để chọn example đa dạng về nội dung, không chỉ về các từ
'''
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(test_df['data'].tolist(), show_progress_bar=True)

# KMeans clustering (70% đa dạng, 30% khó)
n_total = 5
n_diverse = int(n_total * 0.7)
n_hard = n_total - n_diverse

kmeans = KMeans(n_clusters=n_diverse, random_state=42, n_init=10)
kmeans.fit(embeddings)

# Lấy diverse example gần cluster centroid
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
diverse_examples = test_df.iloc[closest]

print(f"Selected {len(diverse_examples)} diverse examples:")
for i, row in diverse_examples.iterrows():
    print(f"\nExample {i}:\n{row['data']}")

Batches: 100%|██████████| 51/51 [00:12<00:00,  4.16it/s]


Selected 3 diverse examples:

Example 1524:
Son đẹp, hàng hịn😁, mê lắm luôn, tuy k lỳ nhưng bù lại bôi lên môi nó không bị khô môi, xinh lắm luôn, đóng gói rất chắc chắn, giá rẻ, giao hàng từ TQ về VN nên hơi lâu...nhưng không sao😊xinh là thích😂😂 nên mua ủng hộ shop nhé m.n

Example 1574:
Màu son đẹp nhưng hơi lâu khô tí... 
Chất son cũng tạm được
Vỏ son đẹp
Son thơm cực luoonnnn

Example 1591:
đẹppppppppppp xĩuuuuuuuuuuuu hdjhsjsjsjdjdjjdjdjdjdjd


In [20]:
all_indices = np.arange(len(test_df))

remaining_indices = list(set(all_indices) - set(closest))

# Chọn ngẫu nhiên n_hard example từ phần còn lại 
hard_examples = test_df.iloc[remaining_indices].sample(n=n_hard, random_state=42)

print(f"Selected {len(hard_examples)} hard examples:")
for i, row in hard_examples.iterrows():
    print(f"\nExample {i}:\n{row['data']}")

Selected 2 hard examples:

Example 135:
son sịn . màu đẹp jaisidiisisjsjsjsjdiididdiidjsjdjjdididi

Example 479:
Giao hàng nhanh chóng. Hơi nhỏ so với tưởng tượng của mình. Z z z nhưng với giá tiền thì chấp nhận ạ. Nên cho 5 sao


In [21]:
def select_few_shot_examples(df, text_column, n_total=5, model_name='all-MiniLM-L6-v2', random_state=42):
    n_diverse = int(n_total * 0.7)
    n_hard = n_total - n_diverse

    model = SentenceTransformer(model_name)
    embeddings = model.encode(df[text_column].tolist(), show_progress_bar=True)

    kmeans = KMeans(n_clusters=n_diverse, random_state=random_state, n_init=10)
    kmeans.fit(embeddings)

    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    diverse_df = df.iloc[closest]

    all_indices = np.arange(len(df))
    remaining_indices = list(set(all_indices) - set(closest))
    hard_df = df.iloc[remaining_indices].sample(n=n_hard, random_state=random_state)

    return diverse_df.reset_index(drop=True), hard_df.reset_index(drop=True)

# SET-UP FEW-SHOT EXAMPLES
diverse, hard = select_few_shot_examples(test_df, text_column='data', n_total=5)

few_shot_json =  transform_aspect_sentiment(diverse, 0, 100) + transform_aspect_sentiment(hard, 0, 100)


Batches: 100%|██████████| 51/51 [00:12<00:00,  4.19it/s]


In [22]:
few_shot_json[:10]

[{'id': '0',
  'text': 'Son đẹp, hàng hịn😁, mê lắm luôn, tuy k lỳ nhưng bù lại bôi lên môi nó không bị khô môi, xinh lắm luôn, đóng gói rất chắc chắn, giá rẻ, giao hàng từ TQ về VN nên hơi lâu...nhưng không sao😊xinh là thích😂😂 nên mua ủng hộ shop nhé m.n',
  'sentiments': [{'aspect': 'stayingpower', 'sentiment': 'negative'},
   {'aspect': 'texture', 'sentiment': 'positive'},
   {'aspect': 'price', 'sentiment': 'positive'},
   {'aspect': 'shipping', 'sentiment': 'negative'},
   {'aspect': 'packing', 'sentiment': 'positive'}]},
 {'id': '1',
  'text': 'Màu son đẹp nhưng hơi lâu khô tí... \nChất son cũng tạm được\nVỏ son đẹp\nSon thơm cực luoonnnn',
  'sentiments': [{'aspect': 'texture', 'sentiment': 'positive'},
   {'aspect': 'smell', 'sentiment': 'positive'},
   {'aspect': 'colour', 'sentiment': 'positive'}]},
 {'id': '2',
  'text': 'đẹppppppppppp xĩuuuuuuuuuuuu hdjhsjsjsjdjdjjdjdjdjdjd',
  'sentiments': [{'aspect': 'others', 'sentiment': 'neutral'}]},
 {'id': '0',
  'text': 'son sịn . m

In [23]:
def evaluate_aspect_sentiment(ground_truth, predictions):
    true_aspects = []
    pred_aspects = []

    true_aspect_sentiments = []
    pred_aspect_sentiments = []

    for gt_entry, pred_entry in zip(ground_truth, predictions):
        gt_sents = gt_entry['sentiments']
        gt_aspect_set = set()
        gt_aspect_sentiment_set = set()

        for item in gt_sents:
            gt_aspect_set.add(item['aspect'])
            gt_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        true_aspects.append(gt_aspect_set)
        true_aspect_sentiments.append(gt_aspect_sentiment_set)

        pred_sents = pred_entry['results']
        pred_aspect_set = set()
        pred_aspect_sentiment_set = set()

        for item in pred_sents:
            pred_aspect_set.add(item['aspect'])
            pred_aspect_sentiment_set.add((item['aspect'], item['sentiment']))

        pred_aspects.append(pred_aspect_set)
        pred_aspect_sentiments.append(pred_aspect_sentiment_set)

    all_true_aspects = set.union(*true_aspects) if true_aspects else set()
    all_pred_aspects = set.union(*pred_aspects) if pred_aspects else set()

    tp_aspect = sum(len(gt & pred) for gt, pred in zip(true_aspects, pred_aspects))
    fp_aspect = sum(len(pred - gt) for gt, pred in zip(true_aspects, pred_aspects))
    fn_aspect = sum(len(gt - pred) for gt, pred in zip(true_aspects, pred_aspects))

    precision_aspect = tp_aspect / (tp_aspect + fp_aspect + 1e-8)
    recall_aspect = tp_aspect / (tp_aspect + fn_aspect + 1e-8)
    f1_aspect = 2 * precision_aspect * recall_aspect / (precision_aspect + recall_aspect + 1e-8)

    tp_sentiment = sum(len(gt & pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fp_sentiment = sum(len(pred - gt) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))
    fn_sentiment = sum(len(gt - pred) for gt, pred in zip(true_aspect_sentiments, pred_aspect_sentiments))

    precision_sentiment = tp_sentiment / (tp_sentiment + fp_sentiment + 1e-8)
    recall_sentiment = tp_sentiment / (tp_sentiment + fn_sentiment + 1e-8)
    f1_sentiment = 2 * precision_sentiment * recall_sentiment / (precision_sentiment + recall_sentiment + 1e-8)

    return {
        "Aspect Detection F1": f1_aspect,
        "Sentiment Classification F1": f1_sentiment
    }

In [24]:
# Setup Grok API
def call_grok_api(prompt):
    url = "https://api.x.ai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {os.getenv('GROK_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "grok-3",
        "messages": [
            {"role": "system", "content": "You are an AI assistant that extracts aspects and their sentiments from text."},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0
    }
    
    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        if response.status_code == 200:
            result = response.json()
            return result['choices'][0]['message']['content']
        else:
            print(f"API Error: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"Request Error: {e}")
        return None

def create_clustering_prompt(text, few_shot_examples):
    prompt = "You are an AI assistant that extracts aspects and their sentiments from text.\n\n"
    
    # Thêm few-shot examples
    for ex in few_shot_examples:
        prompt += f"Extract aspects and sentiments from the following review:\n{ex['text']}\n"
        prompt += f"Result: {json.dumps({'results': ex['sentiments']}, ensure_ascii=False)}\n\n"
    
    # Thêm câu hỏi hiện tại
    prompt += f"Extract aspects and sentiments from the following review:\n{text}\n"
    prompt += f"Available aspects: {', '.join(aspects)}\n"
    prompt += "Available sentiments: positive, negative, neutral\n"
    prompt += "Return ONLY a valid JSON object in this exact format:\n"
    prompt += '{"results": [{"aspect": "aspect_name", "sentiment": "sentiment_value"}]}'
    
    return prompt

def extract_with_grok_clustering(text, few_shot_examples):
    prompt = create_clustering_prompt(text, few_shot_examples)
    response_text = call_grok_api(prompt)
    
    if response_text:
        try:
            # Tìm JSON trong response
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1
            
            if start_idx != -1 and end_idx != -1:
                json_str = response_text[start_idx:end_idx]
                parsed_output = json.loads(json_str)
                return parsed_output
            else:
                return {"results": []}
        except json.JSONDecodeError:
            print(f"JSON Parse Error: {response_text}")
            return {"results": []}
    else:
        return {"results": []}

predictions = []

# Dự đoán
for data in tqdm(test_json):
    prediction = extract_with_grok_clustering(data['text'], few_shot_json)
    predictions.append(prediction)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [03:53<00:00,  2.34s/it]


In [25]:
scores = evaluate_aspect_sentiment(test_json, predictions)
print(scores)

{'Aspect Detection F1': 0.9060240913734939, 'Sentiment Classification F1': 0.785542163668457}


In [27]:
result_file = path.join(RESULT_DIR, 'ViABSA_BP_Few-shot-BP-Grok.json')
with open(result_file, 'w') as f:
    json.dump(predictions, f, indent=4, ensure_ascii=False)